<a href="https://colab.research.google.com/github/projjal1/Realtime-Emotion-Detection/blob/master/Feret_2013_Emotion_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Using Kaggle Emotion Detection Dataset

In [ ]:
!unzip drive/'My Drive'/'Colab Notebooks'/fer2013.zip

Archive:  drive/My Drive/Colab Notebooks/fer2013.zip
  inflating: fer2013.csv             


In [ ]:
!pip install tensorflow-gpu

     |████████████████████████████████| 320.4MB 43kB/s 


Importing all modules

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2 
from tensorflow.keras.layers import Conv2D,BatchNormalization,Activation,AveragePooling2D,Dropout,GlobalAveragePooling2D,SeparableConv2D,MaxPooling2D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential

In [ ]:
#file path name
path_name='fer2013.csv'

Loading the dataset from the CSV file

In [ ]:
#Load the data 
def _load_fer2013():
    data = pd.read_csv(path_name)
    pixels = data['pixels'].tolist()
    width, height = 48, 48
    faces = []
    for pixel_sequence in pixels:
        face = [int(pixel) for pixel in pixel_sequence.split(' ')]
        face = np.asarray(face).reshape(width, height)
        #Set image size to (64,64)
        face = cv2.resize(face.astype('uint8'), (64,64))
        faces.append(face.astype('float32'))
    faces = np.asarray(faces)
    faces = np.expand_dims(faces, -1)
    emotions = pd.get_dummies(data['emotion']).values
    return faces, emotions

#Loading the data from module
faces,emotion=_load_fer2013()

Labels or outcomes of this training

In [ ]:
labels={'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'sad': 4, 'surprise': 5, 'neutral': 6}

Normalizing dataset of images

In [ ]:
#Preprocess the image-data  
faces=faces.astype('float32')
faces=faces/255

In [ ]:
faces[1]

array([[[0.5921569 ],
        [0.5882353 ],
        [0.58431375],
        ...,
        [0.53333336],
        [0.5176471 ],
        [0.47058824]],

       [[0.5921569 ],
        [0.5882353 ],
        [0.58431375],
        ...,
        [0.5254902 ],
        [0.5372549 ],
        [0.5137255 ]],

       [[0.5921569 ],
        [0.5882353 ],
        [0.5882353 ],
        ...,
        [0.5019608 ],
        [0.53333336],
        [0.54901963]],

       ...,

       [[0.7372549 ],
        [0.7372549 ],
        [0.7058824 ],
        ...,
        [0.72156864],
        [0.72156864],
        [0.73333335]],

       [[0.73333335],
        [0.7294118 ],
        [0.7372549 ],
        ...,
        [0.7254902 ],
        [0.72156864],
        [0.7294118 ]],

       [[0.7294118 ],
        [0.7254902 ],
        [0.72156864],
        ...,
        [0.73333335],
        [0.7176471 ],
        [0.72156864]]], dtype=float32)

Shapes of dataset of images and labels

In [ ]:
print("Shape of image data: ",faces.shape)

Shape of image data:  (35887, 64, 64, 1)


In [ ]:
print("Shape of label data: ",emotion.shape)

Shape of label data:  (35887, 7)


Split the data to 70% of training and 30% of validation

In [ ]:
#Split the data
def split_data(x, y, validation_split=.2):
    num_samples = len(x)
    num_train_samples = int((1 - validation_split)*num_samples)
    train_x = x[:num_train_samples]
    train_y = y[:num_train_samples]
    val_x = x[num_train_samples:]
    val_y = y[num_train_samples:]
    train_data = (train_x, train_y)
    val_data = (val_x, val_y)
    return train_data, val_data

In [ ]:
train_data, val_data = split_data(faces, emotion, 0.3)

In [ ]:
train_faces, train_emotions = train_data

Shape of images after splitting

In [ ]:
print(train_faces.shape)

(25120, 64, 64, 1)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Applying all transformations to generate more traning sets

In [ ]:
data_generator = ImageDataGenerator(
                        featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        zoom_range=.1,
                        horizontal_flip=True)

In [ ]:
batch_size = 32
fit_data=data_generator.flow(train_faces, train_emotions,batch_size)

In [ ]:
from tensorflow.keras import Input

In [ ]:
from tensorflow.keras import layers

In [ ]:
from tensorflow.keras import models

A simple CNN model for the training

In [ ]:
model=models.Sequential()
model.add(layers.Conv2D(filters=16, kernel_size=(2, 2), padding="same", activation="relu", input_shape=(64,64,1)))
model.add(layers.MaxPooling2D(pool_size=(2, 2), padding='same'))

model.add(layers.Conv2D(filters=32, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(layers.MaxPooling2D(pool_size=(2, 2), padding='same'))

model.add(layers.Conv2D(filters=64, kernel_size=(4, 4), padding="same", activation="relu"))
model.add(layers.MaxPooling2D(pool_size=(2, 2), padding='same'))

model.add(layers.Conv2D(filters=128, kernel_size=(5, 5), padding="same", activation="relu"))
model.add(layers.MaxPooling2D(pool_size=(2, 2), padding='same'))

model.add(layers.Flatten())

model.add(layers.Dense(1024, activation="relu"))

model.add(layers.Dense(7, activation="softmax"))

Summary of the model

In [ ]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 64, 64, 16)        80        
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 32, 32, 16)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 32, 32, 32)        4640      
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 16, 16, 64)        32832     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 8, 8, 128)        

Compiling with Adam as optimizer and categorical CrossEntropy as loss function

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history=model.fit_generator(fit_data,steps_per_epoch=train_faces.shape[0]//batch_size,epochs=100,validation_data=val_data)

Generating random sets from the datset to get validation accuracy

In [ ]:
from random import randint

correct=0

for each in range(1200):
  r=randint(0,faces.shape[0])
  predict_data=np.expand_dims(faces[r],0)
  pred=np.argmax(model.predict(predict_data))
  actual=np.argmax(emotion[r])
  if pred==actual:
    correct+=1

print('Correctly predicted : ',correct/1200*100,' %')
print('Incorrectly predicted : ',(1200-correct)/1200*100,' %')

Correctly predicted :  78.66666666666666  %
Incorrectly predicted :  21.333333333333336  %


Saving the model 

In [ ]:
model.save('emotion.h5')
model.save('emotion.hdf5')

In [ ]:
model.save('my_model')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: my_model/assets


In [ ]:
model.save_weights('weights')

In [ ]:
!zip -r model.zip my_model

updating: my_model/ (stored 0%)
  adding: my_model/saved_model.pb (deflated 90%)
  adding: my_model/assets/ (stored 0%)
  adding: my_model/variables/ (stored 0%)
  adding: my_model/variables/variables.index (deflated 67%)
  adding: my_model/variables/variables.data-00000-of-00001 (deflated 41%)


Loading the model with weights

In [ ]:
model_new=tf.keras.models.load_model('emotion.h5')

In [ ]:
from random import randint

correct=0

for each in range(1200):
  r=randint(0,faces.shape[0])
  predict_data=np.expand_dims(faces[r],0)
  pred=np.argmax(model_new.predict(predict_data))
  actual=np.argmax(emotion[r])
  if pred==actual:
    correct+=1

print('Correctly predicted : ',correct/1200*100,' %')
print('Incorrectly predicted : ',(1200-correct)/1200*100,' %')

Correctly predicted :  77.0  %
Incorrectly predicted :  23.0  %


In [ ]:
tf.__version__

'2.3.0'